# Fetch Taxi Zone data

In [1]:
import geopandas as gpd
import folium
import pandas as pd
import os
import polyline
import requests
import json
from shapely.geometry.multipolygon import MultiPolygon
from shapely import wkt

import matplotlib.pyplot as plt

root_folder = "/home/trungdc/unimelb/MAST30024/asm/mast30034_2021_s2_project_1-alexdang02-1/"
data_dir = os.path.join(root_folder, "Data","Taxi Zones")


In [2]:
df = gpd.read_file(os.path.join(data_dir, "taxi_zones/taxi_zones.shp"))
zone = pd.read_csv(os.path.join(data_dir,"taxi+_zone_lookup.csv"))
df['geometry'] = df['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
df["Centroid"] = df["geometry"].apply(lambda multipolygon: multipolygon.centroid.wkt)

In [3]:
df.sort_values("LocationID", ascending=True,  inplace = True)
Centroid_long = []
Centroid_la = []
for index, row in df.iterrows():
    processed_Centroid = [float(item) for item in row["Centroid"][7:][:-1].split(" ")]
    Centroid_long.append(processed_Centroid[0])
    Centroid_la.append(processed_Centroid[1])
df["Centroid_long"] = Centroid_long
df["Centroid_la"] = Centroid_la
df.drop("Centroid", axis = 1, inplace=True)
df.to_csv(os.path.join(data_dir, "taxi_zones_processed.csv"))
df.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,Centroid_long,Centroid_la
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695...",-74.174000,40.691831
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",-73.831299,40.616745
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870...",-73.847422,40.864474
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725...",-73.976968,40.723752
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562...",-74.188484,40.552659


# Extract OSMR features

## Intruction on setting up OSMR Backend with Docker on Ubuntu 20.04
Please follow this [repo](https://github.com/Project-OSRM/osrm-backend) if you have any problem

0. Install Docker and go to OSMR directory
```
    cd OSMR_folder
```
1. Download NYC pbf file
    ```
    !wget http://download.geofabrik.de/north-america/us/new-york-latest.osm.pbf
    ```
2. Pre-process the extract 
```
    docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-extract -p /opt/car.lua /data/new-york-latest.osm.pb
```
3. Setting up container
```
    docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-partition /data/new-york-latest.osm.pbf
    docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-customize /data/new-york-latest.osm.pbf
```
4. Start the container on port 5000
```
    docker run -t -i -p 5000:5000 -v "${PWD}:/data" osrm/osrm-backend osrm-routed --algorithm mld /data/new-york-latest.osrm
```
NOTE: make sure the container is running before running the following two cells


In [17]:
def process_OSMR_output(OSMR_Output : json):
    """
    Extracting important feature from OSMR output
        Parameters:
            OSMR_Output (json) : filename of OSRM output
        Returns:
            features (dict): Extracted features including the following fields:
                - geometry: Poliline encoder for visualisation of the route
                - total_distance: distance of nearest route between two Taxi Zone Centroid in meters
                - total_duration: duration of nearest route between two Taxi Zone Centroid in seconds
                - main_road: List of main road names of the route 
                - segment_road: List of road segment in the route has this form 
                    [name of segment, duration of traveling through segment, distance of segment]
                - AVG_speed_OSRM: Average speed through out the route in km/h

    """
    features = {}
    startnode = OSMR_Output["routes"][0]
    features["geometry"] = startnode["geometry"]
    features["total_distance"] = startnode["legs"][0]["distance"]
    features["total_duration"] = startnode["legs"][0]["duration"]
    features["main_road"] = startnode["legs"][0]["summary"].split(",")
    features["segment_road"] = [[routeleg["name"], routeleg["duration"], routeleg["distance"] ] for routeleg in startnode["legs"][0]["steps"]]
    features["AVG_speed_OSRM"] = (features["total_distance"]/1000)/(features["total_duration"]/(60*60))
    return features

In [ ]:
output_OSRM_dataframe = []
for pickupid in df["LocationID"].values:
    for dropoffid in df["LocationID"].values:

        # cannot extract features for cases when pickupid and dropoffid are the same
        if pickupid == dropoffid:
            continue
    
        pickuppos = df[df.LocationID == pickupid]["Processed Centroid"].values[0]
        dropoffpos = df[df.LocationID == dropoffid]["Processed Centroid"].values[0]
        pickup_lon, pickup_lat, dropoff_lon, dropoff_lat = pickuppos[0], pickuppos[1], dropoffpos[0],dropoffpos[1]

        loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
        url = "http://127.0.0.1:5000/route/v1/driving/"
        r = requests.get(url + loc + "?alternatives=false&steps=true&overview=full") 
        if r.status_code!= 200:
            res = {}

        res = r.json()   
        with open(os.path.join(os.path.join(root_folder, "Data", "OSRM/OSRM_Output"), str(pickupid)+ "-"+ str(dropoffid) +'.txt'), 'w')as outfile:
            json.dump(res, outfile)
        
        output = process_OSMR_output(res)
        output["Key"] = f"{pickupid}-{dropoffid}"
        output_OSRM_dataframe.append(output)
pd.DataFrame(output_OSRM_dataframe).to_csv(os.path.join(root_folder,"Data", "OSRM", "OSRM.csv"))

# Demo for OSMR Project

In [11]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://127.0.0.1:5000/route/v1/driving/"
    r = requests.get(url + loc + "?alternatives=false&steps=true&overview=full") 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out
pickup_lon, pickup_lat, dropoff_lon, dropoff_lat = -74.17400027276304,40.69183120640133,-73.83129854302213,40.61674529165957
test_route = get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
test_route
def get_map(route):
    
    m = folium.Map(location=[(route['start_point'][0] + route['end_point'][0])/2, 
                             (route['start_point'][1] + route['end_point'][1])/2], 
                   zoom_start=12)

    folium.PolyLine(
        route['route'],
        weight=8,
        color='blue',
        opacity=0.6
    ).add_to(m)

    folium.Marker(
        location=route['start_point'],
        icon=folium.Icon(icon='play', color='green')
    ).add_to(m)

    folium.Marker(
        location=route['end_point'],
        icon=folium.Icon(icon='stop', color='red')
    ).add_to(m)

    return m
get_map(test_route)

# Import data for cleaning & EDA

In [44]:
df = pd.read_csv(os.path.join(root_folder,"Data", "OSRM", "OSRM.csv"))
df.drop(["Unnamed: 0", "segment_road"], inplace= True, axis = 1)
df.columns = ['geometry', 'expected_total_distance', 'expected_total_duration',"expected_main_road",
       'Expected_AVG_speed', 'Key']
df["expected_total_distance"] = (df["expected_total_distance"]/1000)*0.621371
df.head()

,geometry,expected_total_distance,expected_total_duration,expected_main_road,Expected_AVG_speed,Key
0,ebawFdnxcMl@q@n@_AnKfNnGvIr@cAZe@^i@~HmMVaAjAe...,28.794829,2806.0,"['Staten Island Expressway', ' Belt Parkway']",59.453628,1-2
1,ebawFdnxcMl@q@n@_AnKfNnGvIr@cAZe@^i@~HmMVaAjAe...,38.013303,3823.1,"['Staten Island Expressway', ' Brooklyn-Queens...",57.606497,1-3
2,ebawFdnxcMl@q@n@_AnKfNnGvIr@cAZe@^i@~HmMVaAjAe...,20.909383,2488.2,"['Staten Island Expressway', ' Gowanus Express...",48.686376,1-4
3,ebawFdnxcMl@q@n@_AnKfNnGvIr@cAZe@^i@~HmMVaAjAe...,7.804171,1251.2,"['West Shore Expressway', ' W Shore Exwy W Sta...",36.136957,1-5
4,ebawFdnxcMl@q@n@_AnKfNnGvIr@cAZe@^i@~HmMVaAjAe...,8.161087,1115.9,"['Staten Island Expressway', ' Staten Island E...",42.371539,1-6


,geometry,expected_total_duration,expected_main_road,Expected_AVG_speed,Key,LocationID
0,0,0,0,0,1-1,1
1,0,0,0,0,2-2,2
2,0,0,0,0,3-3,3
3,0,0,0,0,4-4,4
4,0,0,0,0,5-5,5


In [56]:
more_row = []
for id in range(1,264):
    row = { "geometry": 0, "expected_total_duration":0,"expected_main_road": 0
    , "Expected_AVG_speed":0, "Key": str(id)+"-"+str(id), "LocationID": int(id)   }
    more_row.append(row)
more_row = pd.DataFrame(more_row)
# more_row.head()
taxi_zone = gpd.read_file(os.path.join(data_dir, "taxi_zones/taxi_zones.shp"))
taxi_zone.to_csv(os.path.join(data_dir, "taxi_zones/taxi_zones.csv"))
taxi_zone = pd.read_csv(os.path.join(data_dir, "taxi_zones/taxi_zones.csv"))
taxi_zone["LocationID"].astype("int")
taxi_zone= taxi_zone[["LocationID", "Shape_Leng"]]


more_row = more_row.merge(taxi_zone, on="LocationID", how="inner")
more_row.drop("LocationID", axis=1, inplace = True)
more_row.columns =  ['geometry', 'expected_total_duration',"expected_main_road",
       'Expected_AVG_speed', 'Key', "expected_total_distance"]
more_row.head()

,geometry,expected_total_duration,expected_main_road,Expected_AVG_speed,Key,expected_total_distance
0,0,0,0,0,1-1,0.116357
1,0,0,0,0,2-2,0.433470
2,0,0,0,0,3-3,0.084341
3,0,0,0,0,4-4,0.043567
4,0,0,0,0,5-5,0.092146


In [57]:
merge = pd.concat([df,more_row])
merge.head()
merge.to_csv(os.path.join(root_folder,"Data", "OSRM", "OSRM_plus.csv"))
